# FantasticSql

![图1](data/1.png)


## 主程序

In [1]:
from run import FantasticSql

In [2]:
question_dict =  {
        "tid": "tttt----62",
        "team": [
            {
                "id": "tttt----62----35-1-1",
                "question": "博时基金公司成立于（XXXX年XX月XX日）？",
                "answer": "博时基金公司成立于（1998年07月13日）"
            },
            {
                "id": "tttt----62----35-1-2",
                "question": "比博时基金成立更早的基金公司(简称)有哪些？",
                "answer": "比博时基金成立更早的基金公司(简称)有：华夏基金和南方基金。"
            },
            {
                "id": "tttt----62----35-1-3",
                "question": "和博时基金属于同一地区的基金公司(简称)有哪些？",
                "answer": "和博时基金属于同一地区的基金公司(简称)有：鹏华基金和南方基金。"
            }
        ]
    }

In [4]:
FantasticSql(question_dict)

```json
[{"基金公司简称":"博时"}]
```
{"success":true,"data":[{"ChiName":"博时基金管理有限公司","CompanyCode":41509}],"count":1}
```json
[
    {"question":"博时基金公司成立于（XXXX年XX月XX日）？","query_requirements":"查询博时基金公司的成立日期","table_name":"MF_InvestAdvisorOutline"},
    {"question":"比博时基金成立更早的基金公司(简称)有哪些？","query_requirements":"查询所有基金公司的成立日期并进行排序，筛选出成立日期早于博时基金的公司","table_name":"MF_InvestAdvisorOutline"},
    {"question":"和博时基金属于同一地区的基金公司(简称)有哪些？","query_requirements":"查询博时基金公司的地区信息","table_name":"MF_InvestAdvisorOutline"},
    {"question":"和博时基金属于同一地区的基金公司(简称)有哪些？","query_requirements":"查询与博时基金公司同一地区的其他基金公司","table_name":"MF_InvestAdvisorOutline"}
]
```
```sql
SELECT EstablishmentDate 
FROM PublicFundDB.MF_InvestAdvisorOutline 
WHERE InvestAdvisorCode = (
  SELECT CompanyCode 
  FROM InstitutionDB.LC_InstiArchive 
  WHERE ChiName = '博时基金管理有限公司'
);
```
{"success":true,"data":[{"EstablishmentDate":"1998-07-13 12:00:00.000"}],"count":1}
博时基金公司成立于（1998年07月13日）。
```sql
SELECT DISTINCT InvestAdvisorAbbrName
FROM Publ

# 简化说明

In [ ]:

def FantasticSql_simple(question_dict):
    question_list = question_dict['team']
    question_content = '\n'.join([i['question'] for i in question_list])
    # 获取题目中的实体信息，提前获取innercode等信息
    info, tables = process_question(question_content)
    if info:
        process_info = f"实体匹配数据表信息为：{info}"
    else:
        process_info = ''

    # 输入所有表格，并找到题目所需要的表格
    recall_tables = recall_table(question_content)
    db_recall = list(set([i['table_name'] for i in recall_tables]))
    table_desc = '\n'.join([get_table_desc(i,recall_by=question_content,recall_num=20) for i in db_recall])

    # 定义任务，要求写SQL
    messages = [{'role': 'system', 'content': f'''
用户的数据库描述为:
{table_desc}
{process_info}
'''}]

    question_list = question_dict['team']
    question_dict['table'] = recall_tables
    for q in question_list:
        messages.append({'role': 'user', 'content': f"""
请编写sql解决问题：{q['question']}，你有三次机会，你可以一次编写或者分步骤编写，sql在 ```sql ```内。
请仅仅使用我给你的信息，不要自己编造code。
"""})

        for try_num in range(3):
            answer = llm(messages)
            if '```sql' in answer:
                sql_result, sql = exec_sql(answer)
                messages.append({'role': 'user', 'content': f"""sql查询结果为：{sql_result}，
    如果可以回答问题：{q['question']}则回答
    否则继续写sql
    """})
                q['sql'] = sql
            else:
                q['answer'] = process_answer(answer)

                # 每次回答都保存
                with open('submit.json', 'w', encoding='utf8') as f:
                    json.dump(all_question, f)
                break